# Trying Sherpa

I was already making code ready to post a question on StackOverflow, when I found this question and the related answers:

https://stackoverflow.com/questions/15345906/user-specified-function-in-mpfit

The first answer links to the modelling and fitting package `sherpa` which I have never heard of before, but it seems to be super well documented and writen with an object-oriented approach, so I decided to see whether that would actually be useful for us. Here's the link to it:

http://cxc.harvard.edu/contrib/sherpa/

`pip install sherpa` did it.

I started by just copy-pasting the first example from their documentation:

https://sherpa.readthedocs.io/en/4.11.0/quick.html

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

os.chdir('../')
from config import CONFIG_INI
import margmodule as marg

## Write your own model class

First, all the sherpa imports.

In [ ]:
from sherpa.models import model
from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

### Define the model method and user clase

This calculates a model form given parameters *pars* and an independent variable *x*.

**This was moved to `margmodule.py`**

In [ ]:
def _transit_model(pars, x, sh):
    """
    Transit model by Mandel & Agol (2002).
    --------
    Params:

    rl: transit depth in Rp/R_star, unitless
    flux:
    epoch: center of transit in days (MJD)
    inclin: inclination of system in radians
    MsMpR: density of system
    ecc: eccentricity of system
    omega: that other weird angle in a planetary system
    per: period of transit in days
    tzero: first x-array data entry in days (MJD)
    c1, c2, c3, c4: limb darkening parameters (quadratic)
    m_fac: ?
    hstp1, hstp2, hstp3, hstp4: HST period systematic parameters (units?)
    xshift1, xshift2, xshift3, xshift4: shift systematic parameters (units?)

    x: array; input time grid
    sh: array, input shifts
    """

    HSTper = CONFIG_INI.getfloat('constants', 'HST_period') * u.d

    # Define each of the parameters that are read into the fitting routine
    (rl, flux, epoch, inclin, MsMpR, ecc, omega, per, tzero, c1, c2, c3, c4,
     m_fac, hstp1, hstp2, hstp3, hstp4, xshift1, xshift2, xshift3, xshift4) = pars

    # Attaching some units
    x *= u.d
    epoch *= u.d
    inclin *= u.rad
    per *= u.d
    tzero *= u.d

    phase = marg.phase_calc(x, epoch, per)  # Per in days here
    HSTphase = marg.phase_calc(x, tzero, HSTper)

    # Calculate the impact parameter as a function of the planetary phase across the star.
    b0 = marg.impact_param(per.to(u.second), MsMpR, phase, inclin)  # period in sec here, incl in radians, b0 in stellar radii

    # Occultnl would be replaced with BATMAN if possible. The main result we need is the rl - radius ratio
    # The c1-c4 are the non-linear limb-darkening parameters
    # b0 is the impact parameter function and I am not sure how this is handled in BATMAN - need to look into this.
    mulimb0, mulimbf = marg.occultnl(rl, c1, c2, c3, c4, b0)
    systematic_model = marg.sys_model(phase, HSTphase, sh, m_fac, hstp1, hstp2, hstp3, hstp4,
                                 xshift1, xshift2, xshift3, xshift4)

    # model fit to data = transit model * baseline flux (flux0) * systematic model
    model = mulimb0 * flux * systematic_model

    return model


class Transit(model.RegriddableModel1D):
    """Transit model"""

    def __init__(self, name='transit', sh=None):
        self.rl = model.Parameter(name, 'rl', 0.12)
        self.flux = model.Parameter(name, 'flux', 1.)
        self.epoch = model.Parameter(name, 'epoch', 57957.970153390, units='days [MJD]')
        self.inclin = model.Parameter(name, 'inclin', np.deg2rad(87.34635), units='radians')
        self.msmpr = model.Parameter(name, 'msmpr', 2014.1042)
        self.ecc = model.Parameter(name, 'ecc', 0.0, units='degrees')
        self.omega = model.Parameter(name, 'omega', 0.0, units='degrees')
        self.period = model.Parameter(name, 'period', 3.73548535, units='days')
        self.tzero = model.Parameter(name, 'tzero', 557957.859985, units='days [MJD]')
        self.c1 = model.Parameter(name, 'c1', 0)
        self.c2 = model.Parameter(name, 'c2', 0)
        self.c3 = model.Parameter(name, 'c3', 0)
        self.c4 = model.Parameter(name, 'c4', 0)
        self.m_fac = model.Parameter(name, 'm_fac', 0, units='?')
        self.hstp1 = model.Parameter(name, 'hstp1', 0)
        self.hstp2 = model.Parameter(name, 'hstp2', 0)
        self.hstp3 = model.Parameter(name, 'hstp3', 0)
        self.hstp4 = model.Parameter(name, 'hstp4', 0)
        self.xshift1 = model.Parameter(name, 'xshift1', 0)
        self.xshift2 = model.Parameter(name, 'xshift2', 0)
        self.xshift3 = model.Parameter(name, 'xshift3', 0)
        self.xshift4 = model.Parameter(name, 'xshift4', 0)

        self.sh_array = sh   # This is not a model parameter but an extra input to the model, like x is

        model.RegriddableModel1D.__init__(self, name,
                                          (self.rl, self.flux, self.epoch,
                                           self.inclin, self.msmpr, self.ecc,
                                           self.omega, self.period, self.tzero,
                                           self.c1, self.c2, self.c3, self.c4,
                                           self.m_fac, self.hstp1, self.hstp2,
                                           self.hstp3, self.hstp4, self.xshift1,
                                           self.xshift2, self.xshift3, self.xshift4))

    def calc(self, pars, x, *args, **kwargs):
        """Evaluate the model"""
        return _transit_model(pars, x, self.sh_array)

### Define the user model class

All parameters have at least a name and an initial guess. Make sure you don't forget a parameter in the initialization of the super class.

**This was moved to `margmodule.py`**

## Import the data

In [ ]:
# Import data
localDir = CONFIG_INI.get('data_paths', 'local_path')
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'),
                           skiprows=7, unpack=True)

In [ ]:
# Plot the data with matplotlib
plt.scatter(x, y, s=5)
plt.title("Data")

## Create Sherpa data object from your data

In [ ]:
# Create Sherpa data object
data = Data1D('Data', x, y, staterror=err)
print(data)

In [ ]:
# Plot the data with Sherpa
dplot = DataPlot()    # create plot object
dplot.prepare(data)      # prepare data for plotting
dplot.plot()          # plot

## Instantiate the model

In [ ]:
# Define the model
tmodel = Transit(name="testmodel", sh=sh)
print(tmodel)

In [ ]:
# Freeze some parameters
tmodel.inclin.frozen = True
print(tmodel.inclin.frozen)

tmodel.msmpr.frozen = True
tmodel.ecc.frozen = True
tmodel.omega.frozen = True
tmodel.period.frozen = True
tmodel.tzero.frozen = True
tmodel.c1.frozen = True
tmodel.c2.frozen = True
tmodel.c3.frozen = True
tmodel.c4.frozen = True
tmodel.m_fac.frozen = True
tmodel.hstp1.frozen = True
tmodel.hstp2.frozen = True
tmodel.hstp3.frozen = True
tmodel.hstp4.frozen = True
tmodel.xshift1.frozen = True
tmodel.xshift2.frozen = True
tmodel.xshift3.frozen = True
tmodel.xshift4.frozen = True

print(tmodel)

In [ ]:
# Plot the model
mplot = ModelPlot()
mplot.prepare(data, tmodel)
mplot.plot()

## The fitting

### Select the statistics and optimizer

In [ ]:
# Select statistics and optimizer
stat = Chi2()
opt = LevMar()

### Instantiate the fit object and fit

In [ ]:
# Fit
tfit = Fit(data, tmodel, stat=stat, method=opt)    # Instantiate fit object
tres = tfit.fit()     # do the fit
print(tres.format())
if not tres.succeeded: print(tres.message)

In [ ]:
# Plot fit over data
fplot = FitPlot()
mplot.prepare(data, tmodel)
fplot.prepare(dplot, mplot)
fplot.plot()